# Example running and interacting with multiple buildings.

In [2]:
import os
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient
from alfalfa_openadr.oadr_client import OADRClient

url should be for running [Alfalfa deployment](https://github.com/NREL/alfalfa/wiki/Deployment) with **at least two workers**

In [3]:
# Create a new client.
# Update url for remote deployments
ac = AlfalfaClient(url='http://localhost')
dr_client = OADRClient(url='http://localhost:5000')

The model_paths below assume the (acess restricted) CCTwin repo is cloned in the same root as this Alfalfa-notebooks repo with branch `apr22demo` checked out.  If not, you will need to update the model_paths.

In [4]:
# Local paths to models for upload
model_paths = []

# all_models = range(0,317)
# for full experiment use range(0,317)
for i in range(0,2):
    model_paths.append(f'../CCTwin-scripts/bldg_models/sfhome/sfhome.zip')

print(model_paths)

['../CCTwin-scripts/bldg_models/sfhome/sfhome.zip', '../CCTwin-scripts/bldg_models/sfhome/sfhome.zip']


In [7]:
# Upload to Alfalfa 
model_ids = []
for p in model_paths:
    print(p)
    # TODO add error handling
    # TODO send uploads asyncronously. 
    # Currently we have sufficient workers to upload in parallel, but this Python loop is forcing them to run in serial.
    model_ids.append(ac.submit(p))

../CCTwin-scripts/bldg_models/sfhome/sfhome.zip
Desired status: READY		Current status: PREPROCESSING
../CCTwin-scripts/bldg_models/sfhome/sfhome.zip
Desired status: READY		Current status: PREPROCESSING


In [8]:
print(model_ids)

['d125451e-e8dd-11ec-b1ba-acde48001122', 'd3922844-e8dd-11ec-b1ba-acde48001122']


In [9]:
# Define the run parameters.  
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2017, 9, 26, 0, 0, 0)
end_dt = datetime.datetime(2017, 9, 27, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": "true",
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

In [10]:
# Start simulations.  since we're using external clock, will take through warmup and wait for an advance
for i in model_ids:
    print(i)
    ac.start(i, **params)

d125451e-e8dd-11ec-b1ba-acde48001122
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: RUNNING
d3922844-e8dd-11ec-b1ba-acde48001122
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STARTED
Desired status: RUNNING		Current status: STA

### Note that this currently advances models "as fast as possible" and isn't attempting to track with real time.

In [19]:
#run for 1 day  
# timesteps = 1440

# limited timesteps during testing
timesteps = 20

# main loop advances simulation and handles model i/o for each timestep

for i in range(timesteps):
    aggregate_load_for_timestep = 0
    # Load current timestamp from one sim.  
    # note you could avoid an api call by adding 60 to start time each iteration
    t = ac.get_sim_time(model_ids[0])

    # Load current values from each model
    for mid in model_ids:
        outputs = ac.outputs(mid)
        #print(outputs)

        # Note the key here is configured in the uploaded model
        # TODO error handling if not present
        main = outputs['Electric Meter']
        aggregate_load_for_timestep+= main

    signals = dr_client.signals()
    signal_write_value = "0"
    for signal in signals:
        if signal['signalName'] == "simple" and signal['signalType'] == "level":
            signal_write_value = signal['payload']

    for id in model_ids:
        inputs = ac.inputs(id)
        if signal_write_value:
            inputs['AlfalfaTest'] = signal_write_value
            ac.setInputs(id, inputs)
        

    # do somethinig with the aggregate demand across buildings for this timestep
    # note that outside this notebook we also are persisting all outputs by building to InfluxDB each timestep
    print(f"{t}, community aggregate: {aggregate_load_for_timestep}")

    # Advance the models.  
    # as currently implemented, there is some serial/blocking execution here at the web layer 
    # which may add up delays when run for 100s buildings
    ac.advance(model_ids)

2017-09-26 00:00:00, community aggregate: 180740.17593316216
2017-09-26 00:01:00, community aggregate: 133061.25066372845
2017-09-26 00:02:00, community aggregate: 132497.6383576323
2017-09-26 00:03:00, community aggregate: 132070.68354430117
2017-09-26 00:04:00, community aggregate: 131675.87134059786
2017-09-26 00:05:00, community aggregate: 131296.46702833145
2017-09-26 00:06:00, community aggregate: 130925.16548272932
2017-09-26 00:07:00, community aggregate: 130557.5221368968
2017-09-26 00:08:00, community aggregate: 130194.66224006668
2017-09-26 00:09:00, community aggregate: 129832.20912225304
2017-09-26 00:10:00, community aggregate: 129474.4253060012
2017-09-26 00:11:00, community aggregate: 129118.5947361834
2017-09-26 00:12:00, community aggregate: 128765.21904689237
2017-09-26 00:13:00, community aggregate: 128413.50897679394
2017-09-26 00:14:00, community aggregate: 128064.54253924207
2017-09-26 00:15:00, community aggregate: 127717.29648353787
2017-09-26 00:16:00, communi

In [20]:
# Stop the models
for id in model_ids:
    ac.stop(id)

Desired status: COMPLETE		Current status: STOPPING
Desired status: COMPLETE		Current status: COMPLETE
Desired status: COMPLETE		Current status: STOPPING
Desired status: COMPLETE		Current status: COMPLETE
